In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc
import circstat as CS
import warnings
warnings.filterwarnings("ignore")

## Helper Functions

In [240]:
def angular_disp(x,y): 
    possible_angles = np.asarray([y-x, y-x+360, y-x-360])
    idxMinAbsAngle = np.abs([y-x, y-x+360, y-x-360]).argmin(axis=0)
    smallest_angle = np.asarray([possible_angles[idxMinAbsAngle[i],i] for i in range(len(possible_angles[0]))])
    return smallest_angle

def get_angle_displacement(df, inp, outp): # different from other deltas - need shortest path
    df = df.sort_values('frame')
    angle = np.array(df[inp])
    a = angular_disp(angle[0:len(angle)-1], angle[1:len(angle)])
    df[outp] = np.concatenate((np.asarray([0]),a))
    return df

def smooth_dyn(df, inp, outp, win):
    fps = df['fps'].unique()[0]
    win = int(win*fps)
    x = df[inp].interpolate()
    df[outp] = x.rolling(window=win,center=False).mean()
    return df

def get_delta(df, inp, outp):
    x = df[inp]
    df[outp]  = np.concatenate((np.asarray([0]),np.diff(x)))*(np.asarray(x*0)+1)
    return df

def get_dynamics_xy_new(xdf, delta_window):
    # Ensure required columns are present
    required_columns = ['video', 'frame', 'x', 'y', 'bp', 'fps', 'pixel_x', 'pixel_y', 'time', 'delta_t', 'part_idx']
    missing_columns = [col for col in required_columns if col not in xdf.columns]
    if missing_columns:
        raise KeyError(f"Missing columns in get_dynamics_xy: {missing_columns}")

    # Calculate dynamics for x/y positions
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: get_delta(x, 'x', 'd_x')).reset_index(drop=True)
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: get_delta(x, 'y', 'd_y')).reset_index(drop=True)
    xdf['total_displacement'] = np.sqrt(xdf['d_x']**2 + xdf['d_y']**2)
    xdf['velocity_x_raw'] = xdf['d_x'] / xdf['delta_t']
    xdf['velocity_y_raw'] = xdf['d_y'] / xdf['delta_t']
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'velocity_x_raw', 'velocity_x', delta_window)).reset_index(drop=True)
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'velocity_y_raw', 'velocity_y', delta_window)).reset_index(drop=True)
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: get_delta(x, 'velocity_x', 'delta_velocity_x')).reset_index(drop=True)
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: get_delta(x, 'velocity_y', 'delta_velocity_y')).reset_index(drop=True)
    xdf['acceleration_x_raw'] = xdf['delta_velocity_x'] / xdf['delta_t']
    xdf['acceleration_y_raw'] = xdf['delta_velocity_y'] / xdf['delta_t']
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'acceleration_x_raw', 'acceleration_x', delta_window)).reset_index(drop=True)
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'acceleration_y_raw', 'acceleration_y', delta_window)).reset_index(drop=True)
    #xdf['acceleration_x2'] = xdf['acceleration_x']**2
    #xdf['acceleration_y2'] = xdf['acceleration_y']**2
    xdf['speed_raw'] = xdf['total_displacement'] / xdf['delta_t']
    xdf = xdf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'speed_raw', 'speed', delta_window)).reset_index(drop=True)
    xdf['part'] = xdf.bp.str[1:]
    xdf['side'] = xdf.bp.str[:1]
    return xdf

def get_dynamics_angle_new(adf, delta_window):
    # Ensure required columns are present
    required_columns = ['video', 'frame', 'angle', 'bp', 'delta_t']
    missing_columns = [col for col in required_columns if col not in adf.columns]
    if missing_columns:
        raise KeyError(f"Missing columns in get_dynamics_angle: {missing_columns}")

    # Calculate dynamics for angles
    adf = adf.groupby(['bp', 'video']).apply(lambda x: get_angle_displacement(x, 'angle', 'angular_displacement')).reset_index(drop=True)
    adf['velocity_raw'] = adf['angular_displacement'] / adf['delta_t']
    adf = adf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'velocity_raw', 'angular_velocity', delta_window)).reset_index(drop=True)
    adf = adf.groupby(['bp', 'video']).apply(lambda x: get_delta(x, 'angular_velocity', 'delta_angular_velocity')).reset_index(drop=True)
    adf['acceleration_raw'] = adf['delta_angular_velocity'] / adf['delta_t']
    adf = adf.groupby(['bp', 'video']).apply(lambda x: smooth_dyn(x, 'acceleration_raw', 'angular_acceleration', delta_window)).reset_index(drop=True)
    #adf['acceleration2'] = adf['acceleration']**2
    adf['part'] = adf.bp.str[1:]
    adf['side'] = adf.bp.str[:1]
    return adf

def get_joint_angles_new(df):
    ''' Compute joint angles from x, y coordinates and retain additional columns '''
    
    # Filter out rows with null x values
    df = df[~df['x'].isnull()]
    
    # Calculate delta_t if fps is present
    if 'fps' in df.columns:
        df['delta_t'] = 1 / df['fps']
    
    # Define body parts and joint connections for angle calculations
    bps = ["Nose", "Neck", "RShoulder", "RElbow", "RWrist", "LShoulder", "LElbow", "LWrist", "RHip", "RKnee", "RAnkle", "LHip", "LKnee",
           "LAnkle", "REye", "LEye", "REar", "LEar"]
    joints = [
        ['part0', 'part1', 'part2'],
        ['RShoulder', 'Neck', 'RElbow'],
        ['RElbow', 'RShoulder', 'RWrist'],
        ['RHip', 'LHip', 'RKnee'],
        ['RKnee', 'RHip', 'RAnkle'],
        ['LShoulder', 'Neck', 'LElbow'],
        ['LElbow', 'LShoulder', 'LWrist'],
        ['LHip', 'RHip', 'LKnee'],
        ['LKnee', 'LHip', 'LAnkle']
    ]
    headers = joints.pop(0)
    df_joints = pd.DataFrame(joints, columns=headers).reset_index()
    df_joints['bpindex'] = df_joints['index'] + 1

    # Merge df with joint definitions to calculate angles
    df_1 = df.copy()
    for i in range(3):
        df_joints['bp'] = df_joints[f'part{i}']
        df_1 = pd.merge(df_1, df_joints[['bp', 'bpindex']], on='bp', how='left')
        df_1[f'idx{i}'] = df_1['bpindex']
        df_1 = df_1.drop('bpindex', axis=1)
        df_1[f'x{i}'] = df_1['x'] * df_1[f'idx{i}'] / df_1[f'idx{i}']
        df_1[f'y{i}'] = df_1['y'] * df_1[f'idx{i}'] / df_1[f'idx{i}']

    # Prepare dataframes for angle calculations
    df0 = df_1[['video', 'frame', 'idx0', 'x0', 'y0', 'bp', 'x', 'y', 'pixel_x', 'pixel_y', 'part_idx']]
    df0 = df0.rename(columns={"bp": "bp0", "idx0": "idx"}).dropna(subset=['idx'])
    df1 = df_1[['video', 'frame', 'idx1', 'x1', 'y1', 'bp']]
    df1 = df1.rename(columns={"bp": "bp1", "idx1": "idx"}).dropna(subset=['idx'])
    df2 = df_1[['video', 'frame', 'idx2', 'x2', 'y2', 'bp']]
    df2 = df2.rename(columns={"bp": "bp2", "idx2": "idx"}).dropna(subset=['idx'])
    df_2 = pd.merge(df0, df1, on=['video', 'frame', 'idx'], how='inner')
    df_2 = pd.merge(df_2, df2, on=['video', 'frame', 'idx'], how='inner')

    # Calculate angles based on x, y coordinates
    df_2['dot'] = (df_2['x1'] - df_2['x0']) * (df_2['x2'] - df_2['x0']) + (df_2['y1'] - df_2['y0']) * (df_2['y2'] - df_2['y0'])
    df_2['det'] = (df_2['x1'] - df_2['x0']) * (df_2['y2'] - df_2['y0']) - (df_2['y1'] - df_2['y0']) * (df_2['x2'] - df_2['x0'])
    df_2['angle_degs'] = np.degrees(np.arctan2(df_2['det'], df_2['dot']))

    # Extract side (L/R) and part (e.g., Shoulder, Elbow) information
    df_2['side'] = df_2['bp0'].str[:1]
    df_2['part'] = df_2['bp0'].str[1:]
    df_2['angle'] = df_2['angle_degs']
    
    # Adjust angles for specific joints
    df_2.loc[df_2['bp0'].isin(['LShoulder', 'LHip']), 'angle'] *= -1
    df_2.loc[df_2['part'].isin(['Elbow', 'Knee']), 'angle'] = np.abs(df_2.loc[df_2['part'].isin(['Elbow', 'Knee']), 'angle'])
    df_2.loc[((df_2['part'] == 'Shoulder') | (df_2['part'] == 'Hip')) & (df_2['angle'] < 0), 'angle'] += 360

    # Add back metadata (e.g., fps, time, delta_t) and retain additional columns
    df_info = df.groupby(['video', 'frame', 'fps', 'time', 'delta_t'], as_index=False).mean(numeric_only=True)[['video', 'frame', 'fps', 'time', 'delta_t']]
    df_angle = pd.merge(
        df_2[['video', 'frame', 'bp0', 'side', 'part', 'angle', 'x', 'y', 'pixel_x', 'pixel_y', 'part_idx']],
        df_info,
        on=['video', 'frame'],
        how='inner'
    ).drop_duplicates()

    # Rename 'bp0' to 'bp' for consistency and return the final DataFrame
    return df_angle.rename(columns={'bp0': 'bp'})


## Loading and Processing Movement Features

In [241]:
pose_estimates_path = '/workspaces/wiggle-face/data-ioana/PANDA1/pose_estimates_r_AIM1_newest.csv'
pose_estimates = pd.read_csv(pose_estimates_path)
print(pose_estimates.shape)
pose_estimates.head()

(7477596, 12)


,video_number,video,bp,frame,x,y,c,fps,pixel_x,pixel_y,time,part_idx
0,0,r_2023_01_18_833180_056_cam3_vid4,Nose,0,1016.508309,324.740814,0.952638,59.94,1920.0,1080.0,0.0,0
1,0,r_2023_01_18_833180_056_cam3_vid4,Neck,0,1054.470492,413.626907,0.919039,59.94,1920.0,1080.0,0.0,1
2,0,r_2023_01_18_833180_056_cam3_vid4,RShoulder,0,1003.232961,407.317051,0.933992,59.94,1920.0,1080.0,0.0,2
3,0,r_2023_01_18_833180_056_cam3_vid4,RElbow,0,947.555373,436.225617,0.935511,59.94,1920.0,1080.0,0.0,3
4,0,r_2023_01_18_833180_056_cam3_vid4,RWrist,0,964.197967,377.496585,0.959464,59.94,1920.0,1080.0,0.0,4


In [242]:
print(pose_estimates.shape)
pose_estimates.head()

(7477596, 12)


,video_number,video,bp,frame,x,y,c,fps,pixel_x,pixel_y,time,part_idx
0,0,r_2023_01_18_833180_056_cam3_vid4,Nose,0,1016.508309,324.740814,0.952638,59.94,1920.0,1080.0,0.0,0
1,0,r_2023_01_18_833180_056_cam3_vid4,Neck,0,1054.470492,413.626907,0.919039,59.94,1920.0,1080.0,0.0,1
2,0,r_2023_01_18_833180_056_cam3_vid4,RShoulder,0,1003.232961,407.317051,0.933992,59.94,1920.0,1080.0,0.0,2
3,0,r_2023_01_18_833180_056_cam3_vid4,RElbow,0,947.555373,436.225617,0.935511,59.94,1920.0,1080.0,0.0,3
4,0,r_2023_01_18_833180_056_cam3_vid4,RWrist,0,964.197967,377.496585,0.959464,59.94,1920.0,1080.0,0.0,4


In [ ]:
required_columns = ['video', 'frame', 'x', 'y', 'pixel_x', 'pixel_y', 'fps', 'time', 'part_idx', 'bp']
missing_columns = [col for col in required_columns if col not in pose_estimates.columns]
if missing_columns:
    raise ValueError(f"Missing columns in pose_estimates: {missing_columns}")
else:
    print("All required columns are present in pose_estimates.")

# compute joint angles to add 'angle', 'side', and 'part' columns
pose_estimates_with_angles = get_joint_angles_new(pose_estimates)

# calculate dynamics for x/y coordinates and angles
delta_window = 2/59.94  # Adjust as needed based on fps
xdf = get_dynamics_xy_new(pose_estimates_with_angles, delta_window)
xdf = xdf.dropna()
xdf = xdf.reset_index(drop = True)
adf = get_dynamics_angle_new(pose_estimates_with_angles, delta_window)
adf = adf.dropna()
adf = adf.reset_index(drop = True)

xdf = xdf.drop(columns = ['acceleration_x_raw', 'acceleration_y_raw', 'velocity_x_raw', 'velocity_y_raw', 'speed_raw'])
adf = adf.drop(columns = ['velocity_raw', 'acceleration_raw'])


movement_feature_df = pd.merge(xdf, adf,how = 'inner',on = ['video', 'bp', 'frame','part', 'part_idx', 'time',
                                                            'side','x', 'y', 'pixel_x', 'pixel_y', 'fps', 'delta_t', 'angle'])
movement_feature_df = movement_feature_df[
    ['video', 'frame', 'bp', 'side', 'part', 'part_idx',  # Basic Information
     'fps', 'time', 'delta_t',                            # Metadata
     'angle',                                             # Angle Information
     'x', 'y', 'pixel_x', 'pixel_y',                      # Position Data
     'd_x', 'd_y', 'total_displacement',                  # Position Displacement
     'velocity_x', 'velocity_y',                          # Velocity
     'speed', 'angular_displacement',                     # Speed
     'angular_velocity', 'delta_angular_velocity',        # Angular Velocity Dynamics
     'delta_velocity_x', 'delta_velocity_y',              # Change in Velocity
     'acceleration_x', 'acceleration_y',                  # Acceleration
     'angular_acceleration']                              # Angular Acceleration
]
movement_feature_df_final = movement_feature_df.drop(columns = ['side','part','delta_t'])
print(movement_feature_df_final.shape)
print(movement_feature_df_final.columns)


All required columns are present in pose_estimates.
(3321936, 25)
Index(['video', 'frame', 'bp', 'part_idx', 'fps', 'time', 'angle', 'x', 'y',
       'pixel_x', 'pixel_y', 'd_x', 'd_y', 'total_displacement', 'velocity_x',
       'velocity_y', 'speed', 'angular_displacement', 'angular_velocity',
       'delta_angular_velocity', 'delta_velocity_x', 'delta_velocity_y',
       'acceleration_x', 'acceleration_y', 'angular_acceleration'],
      dtype='object')


In [ ]:
save_path_1 = '/workspaces/wiggle-face/src/code_for_processing_movement_data/movement_features_per_frame_panda_1.csv'
movement_feature_df_final.to_csv(save_path_1)

## Statistics per time window

In [245]:
# helper functions
def angle_features(df):
    df = df.replace([np.inf, -np.inf], np.nan)
    # - absolute angle
    a_mean = np.degrees(CS.nanmean(np.array(np.radians(df['angle']))))
    # - variability of angle
    a_stdev = np.sqrt(np.degrees(CS.nanvar(np.array(np.radians(df['angle'])))))
    # - measure of complexity (entropy)
    a_ent = ent(df['angle'].round())
    # - median absolute velocity
    median_vel = (np.abs(df['angular_velocity'])).median()
    # - variability of velocity
    IQR_vel = (df['angular_velocity']).quantile(.75) - (df['angular_velocity']).quantile(.25)
    # - variability of acceleration
    IQR_acc = df['angular_acceleration'].quantile(.75) - df['angular_acceleration'].quantile(.25)

    return pd.DataFrame.from_dict({'video':np.unique(df.video),'bp':np.unique(df.bp),\
    'mean_angle':a_mean, 'stdev_angle':a_stdev, 'entropy_angle':a_ent,
    'median_vel_angle':median_vel,'IQR_vel_angle':IQR_vel,\
    'IQR_acc_angle': IQR_acc})

def xy_features(df):
    # - absolute position/angle    
    median_x = df['x'].median()
    median_y = df['y'].median()
    IQR_x = df['x'].quantile(.75)-df['x'].quantile(.25)
    IQR_y = df['y'].quantile(.75)-df['y'].quantile(.25)
    # - median absolute velocity
    median_vel_x = np.abs(df['velocity_x']).median()
    median_vel_y = np.abs(df['velocity_y']).median()
    # - variability of velocity
    IQR_vel_x = df['velocity_x'].quantile(.75)-df['velocity_x'].quantile(.25)
    IQR_vel_y = df['velocity_y'].quantile(.75)-df['velocity_y'].quantile(.25)
    
    # - variability of acceleration
    IQR_acc_x = df['acceleration_x'].quantile(.75) - df['acceleration_x'].quantile(.25)
    IQR_acc_y = df['acceleration_y'].quantile(.75) - df['acceleration_y'].quantile(.25)
    
    # - measure of complexity (entropy)
    ent_x = ent(df['x'].round(2))
    ent_y = ent(df['y'].round(2))
    mean_ent = (ent_x+ent_y)/2
    # define part and side here
    return pd.DataFrame.from_dict({'video':np.unique(df.video),'bp':np.unique(df.bp),\
    'medianx': median_x, 'mediany': median_y, 'IQRx': IQR_x,'IQRy': IQR_y,\
    'medianvelx':median_vel_x, 'medianvely':median_vel_y,\
    'IQRvelx':IQR_vel_x,'IQRvely':IQR_vel_y,\
    'IQRaccx':IQR_acc_x,'IQRaccy':IQR_acc_y,'meanent':mean_ent})

def ent(data):
    p_data = data.value_counts() / len(data)  # probabilities
    entropy = sc.entropy(p_data)
    return entropy

def corr_lr(df, var):
    idf = pd.DataFrame()
    idf['R'] = df[df.side=='R'].reset_index()[var]
    idf['L'] = df[df.side=='L'].reset_index()[var]
    return idf.corr().loc['L','R']

In [246]:
movement_feature_df

,video,frame,bp,side,part,part_idx,fps,time,delta_t,angle,...,velocity_y,speed,angular_displacement,angular_velocity,delta_angular_velocity,delta_velocity_x,delta_velocity_y,acceleration_x,acceleration_y,angular_acceleration
0,r_2023_01_18_833180_056_cam3_vid4,3,RShoulder,R,Shoulder,2,59.94,0.050050,0.016683,145.377442,...,-2.599444,22.630309,-0.736656,-19.863899,-37.055571,7.858894,-12.670538,475.178919,-263.679924,-1044.211460
1,r_2023_01_18_833180_056_cam3_vid4,3,RElbow,R,Elbow,3,59.94,0.050050,0.016683,47.686830,...,-1.235480,14.997973,0.363685,14.495961,-2.998569,9.149304,6.355716,299.476651,238.541245,17.914343
2,r_2023_01_18_833180_056_cam3_vid4,3,LShoulder,L,Shoulder,5,59.94,0.050050,0.016683,128.139960,...,34.699989,35.877242,0.807992,32.654216,42.572966,-16.314653,22.624111,-359.581533,922.765997,1528.819085
3,r_2023_01_18_833180_056_cam3_vid4,3,LElbow,L,Elbow,6,59.94,0.050050,0.016683,21.991766,...,16.988150,26.213407,0.553196,10.107301,35.166010,5.891521,-0.463331,490.268982,49.622730,859.959940
4,r_2023_01_18_833180_056_cam3_vid4,3,RHip,R,Hip,8,59.94,0.050050,0.016683,131.935567,...,-9.297050,27.783004,0.094072,-5.218973,-1.674447,6.518930,1.856582,71.778406,241.253795,-291.091174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321931,r_2023_07_04_833180_074_cam3_vid4,4440,LElbow,L,Elbow,6,59.94,74.074074,0.016683,6.038809,...,-42.505153,49.051690,1.030518,1.618309,40.133253,-39.450138,28.158739,-974.329993,375.503944,-74.099006
3321932,r_2023_07_04_833180_074_cam3_vid4,4440,RHip,R,Hip,8,59.94,74.074074,0.016683,148.396669,...,99.205803,104.076920,1.696600,66.389300,55.089295,-57.047211,100.170611,-1325.203289,5504.082064,2956.727750
3321933,r_2023_07_04_833180_074_cam3_vid4,4440,RKnee,R,Knee,9,59.94,74.074074,0.016683,69.550540,...,-44.653215,58.627138,-1.273751,-44.902816,-27.947870,-8.187983,-19.308089,827.534778,-205.810053,-1464.771452
3321934,r_2023_07_04_833180_074_cam3_vid4,4440,LHip,L,Hip,11,59.94,74.074074,0.016683,130.228147,...,86.383673,93.940319,0.230217,-8.057653,12.012525,-47.226763,77.332302,-879.207816,3742.106215,261.245722


In [247]:
# This code take windows of "time_windows" lenght and caluclates the statistics above for each window

time_window = 2  # seconds
fps = movement_feature_df_final['fps'].iloc[0]  # assuming fps is constant across rows
delta_frames = int(fps * time_window)  # number of frames per 2-second window

# lists to store results
all_xy_features = []
all_angle_features = []

# process each video and body part separately
for (video, bp), group in movement_feature_df_final.groupby(['video', 'bp']):
    # sort by frame to ensure time order
    group = group.sort_values(by='frame').reset_index(drop=True)
    
    # sliding window for 2-second intervals
    window_number = 0
    for start in range(0, len(group), delta_frames):
        # def window
        window = group.iloc[start:start + delta_frames]
        
        if len(window) < delta_frames:
            continue  # Skip incomplete windows at the end of the video
        
        # start time for the window
        start_time = window['frame'].iloc[0] / fps
        
        # xy and angle features for the window
        xy_feats = xy_features(window)
        angle_feats = angle_features(window)
        
        # add back to results
        xy_feats['video'] = video
        xy_feats['bp'] = bp
        xy_feats['time'] = start_time
        xy_feats['window_number'] = window_number
        
        angle_feats['video'] = video
        angle_feats['bp'] = bp
        angle_feats['time'] = start_time
        angle_feats['window_number'] = window_number
        
        # increment window number for each window
        window_number += 1

        # append results
        all_xy_features.append(xy_feats)
        all_angle_features.append(angle_feats)


xy_features_df = pd.concat(all_xy_features, ignore_index=True)
angle_features_df = pd.concat(all_angle_features, ignore_index=True)

# merge xy and angle features into a single DataFrame and Sort by 'video' and 'window_number'
merged_features_df = pd.merge(xy_features_df, angle_features_df, on=['video', 'bp', 'time', 'window_number'], how='outer')
merged_features_df = merged_features_df.sort_values(by=['video', 'window_number']).reset_index(drop=True)

# display the sorted DataFrame
print(merged_features_df.head(20))


                                video         bp      medianx     mediany  \
0   r_2021_07_21_833180_003_cam3_vid4     LElbow  1303.180234  505.291012   
1   r_2021_07_21_833180_003_cam3_vid4       LHip  1201.656615  706.607637   
2   r_2021_07_21_833180_003_cam3_vid4      LKnee  1237.187978  895.712051   
3   r_2021_07_21_833180_003_cam3_vid4  LShoulder  1240.363019  358.961985   
4   r_2021_07_21_833180_003_cam3_vid4     RElbow   915.217890  472.728780   
5   r_2021_07_21_833180_003_cam3_vid4       RHip  1042.024361  705.550345   
6   r_2021_07_21_833180_003_cam3_vid4      RKnee   999.458684  898.214994   
7   r_2021_07_21_833180_003_cam3_vid4  RShoulder  1006.216593  362.502876   
8   r_2021_07_21_833180_003_cam3_vid4     LElbow  1308.904632  506.425223   
9   r_2021_07_21_833180_003_cam3_vid4       LHip  1198.877730  712.624271   
10  r_2021_07_21_833180_003_cam3_vid4      LKnee  1224.671623  910.557807   
11  r_2021_07_21_833180_003_cam3_vid4  LShoulder  1237.581605  360.887935   

In [ ]:
save_path_2 = '/workspaces/wiggle-face/src/code_for_processing_movement_data/movement_statistics_2s_window_panda_1.csv'
merged_features_df.to_csv(save_path_2)